### Date Handling

Also includes performing LEFT JOINS

In [1]:
# Consolidate all imports here
import pandas as pd
import numpy as np
import datetime as dt
from collections import Counter

In [2]:
# Generate weekly calendar
calendar = pd.DataFrame({
    'Week Range': pd.date_range(start='2020-09-01', end=dt.datetime.now()).to_period('W').unique()
})

calendar[['Week Start', 'Week End']] = calendar['Week Range'].astype(str).str.split('/', expand=True)
calendar['Week Start'] = pd.to_datetime(calendar['Week Start'])
calendar.reset_index(inplace=True)
calendar.rename(columns={'index':'Week Number'}, inplace=True)
calendar['Week Number'] = calendar['Week Number'] + 1
calendar = calendar[['Week Range', 'Week Start', 'Week End', 'Week Number']]
calendar.sort_values(by='Week Start', ascending=False, inplace=True)
calendar.reset_index(drop=True, inplace=True)
# calendar.rename(columns={'Week Start':'Date Pulled'}, inplace=True)
calendar['Week End'] = pd.to_datetime(calendar['Week End'])
# calendar['Date Pulled Year'] = calendar['Date Pulled'].dt.year
# calendar = calendar[['Week Range', 'Date Pulled', 'Date Pulled Year', 'Week End', 'Week Number']]
calendar

,Week Range,Week Start,Week End,Week Number
0,2022-03-28/2022-04-03,2022-03-28,2022-04-03,83
1,2022-03-21/2022-03-27,2022-03-21,2022-03-27,82
2,2022-03-14/2022-03-20,2022-03-14,2022-03-20,81
3,2022-03-07/2022-03-13,2022-03-07,2022-03-13,80
4,2022-02-28/2022-03-06,2022-02-28,2022-03-06,79
...,...,...,...,...
78,2020-09-28/2020-10-04,2020-09-28,2020-10-04,5
79,2020-09-21/2020-09-27,2020-09-21,2020-09-27,4
80,2020-09-14/2020-09-20,2020-09-14,2020-09-20,3
81,2020-09-07/2020-09-13,2020-09-07,2020-09-13,2


In [3]:
# Create a dataframe of data to join on the calendar
df = pd.DataFrame({
    'Date Pulled': ['2021-11-29', '2021-12-07', '2021-12-13', '2021-12-20', '2021-12-27', '2022-01-03', '2022-01-10', '2022-01-17', '2022-01-25', '2022-01-31', '2022-02-07', '2022-02-15'],
    'Data': ['*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*', '*']
})

df['Date Pulled'] = pd.to_datetime(df['Date Pulled'])
df

,Date Pulled,Data
0,2021-11-29,*
1,2021-12-07,*
2,2021-12-13,*
3,2021-12-20,*
4,2021-12-27,*
5,2022-01-03,*
6,2022-01-10,*
7,2022-01-17,*
8,2022-01-25,*
9,2022-01-31,*


In [4]:
# # Left join df and calendar
# df_merge = df.merge(calendar, how='left', on='Date Pulled')
# df_merge['Date Pulled Year'] = df_merge['Date Pulled'].dt.year
# df_merge = df_merge[['Date Pulled', 'Date Pulled Year', 'Week End', 'Week Number']]
# df_merge['Week End'] = pd.to_datetime(df_merge['Week End'])
# display(df_merge)
# # display(df_merge[df_merge['Week Number'].isnull()])

In [5]:
# Generate the weekly period based on Date Pulled
df2 = df.copy()
print(df2.dtypes)
df2['Week Range'] = df2['Date Pulled'].dt.to_period('W')
df2

Date Pulled    datetime64[ns]
Data                   object
dtype: object


,Date Pulled,Data,Week Range
0,2021-11-29,*,2021-11-29/2021-12-05
1,2021-12-07,*,2021-12-06/2021-12-12
2,2021-12-13,*,2021-12-13/2021-12-19
3,2021-12-20,*,2021-12-20/2021-12-26
4,2021-12-27,*,2021-12-27/2022-01-02
5,2022-01-03,*,2022-01-03/2022-01-09
6,2022-01-10,*,2022-01-10/2022-01-16
7,2022-01-17,*,2022-01-17/2022-01-23
8,2022-01-25,*,2022-01-24/2022-01-30
9,2022-01-31,*,2022-01-31/2022-02-06


In [6]:
# Merge on whole week-range
df_merge2 = df2.merge(calendar, how='left', on='Week Range')
df_merge2

,Date Pulled,Data,Week Range,Week Start,Week End,Week Number
0,2021-11-29,*,2021-11-29/2021-12-05,2021-11-29,2021-12-05,66
1,2021-12-07,*,2021-12-06/2021-12-12,2021-12-06,2021-12-12,67
2,2021-12-13,*,2021-12-13/2021-12-19,2021-12-13,2021-12-19,68
3,2021-12-20,*,2021-12-20/2021-12-26,2021-12-20,2021-12-26,69
4,2021-12-27,*,2021-12-27/2022-01-02,2021-12-27,2022-01-02,70
5,2022-01-03,*,2022-01-03/2022-01-09,2022-01-03,2022-01-09,71
6,2022-01-10,*,2022-01-10/2022-01-16,2022-01-10,2022-01-16,72
7,2022-01-17,*,2022-01-17/2022-01-23,2022-01-17,2022-01-23,73
8,2022-01-25,*,2022-01-24/2022-01-30,2022-01-24,2022-01-30,74
9,2022-01-31,*,2022-01-31/2022-02-06,2022-01-31,2022-02-06,75


In [7]:
# Validating to_period('W') method
date_now = pd.to_datetime(dt.datetime.now())
display(date_now)
display(date_now.to_period('W'))

Timestamp('2022-03-28 22:36:12.750414')

Period('2022-03-28/2022-04-03', 'W-SUN')

Calendar with Weekly Numbering

In [8]:
from datetime import date

calendars = {}

years = [year for year in range(dt.date(2019, 9, 1).year, dt.datetime.now().year + 5)]

for year in years:
    calendars[f'fiscal_year_{year}'] = pd.DataFrame({
        'Week': pd.date_range(start=f'{year - 1}-09-01', end=f'{year}-08-31').to_period('W-SAT').unique(),
    })

In [9]:
for df in calendars:
    calendars[df][['Week Start', 'Week End']] = calendars[df]['Week'].astype(str).str.split('/', expand=True)
    calendars[df]['Calendar Week Number'] = calendars[df]['Week'].dt.week
    calendars[df]['Week Start'] = pd.to_datetime(calendars[df]['Week Start'])
    calendars[df]['Week End'] = pd.to_datetime(calendars[df]['Week End'])
    calendars[df]['Week Start Month Number'] = calendars[df]['Week Start'].dt.month

    if calendars[df].iloc[[0]]['Week Start Month Number'].item() != 9:
        calendars[df].drop(calendars[df].iloc[[0]].index.item(), axis=0, inplace=True)
    else:
        pass

    if calendars[df].iloc[[-1]]['Week Start Month Number'].item() != 8:
        calendars[df].drop(calendars[df].iloc[[-1]].index.item(), axis=0, inplace=True)
    else:
        pass

    calendars[df].reset_index(drop=True, inplace=True)
    calendars[df].reset_index(inplace=True)
    calendars[df].rename(columns={'index':'Week Sort Number'}, inplace=True)
    calendars[df].sort_values(by='Week Start', ascending=False, inplace=True)
    calendars[df]['Week Sort Number'] = calendars[df]['Week Sort Number'] + 1
    calendars[df].reset_index(drop=True, inplace=True)
    calendars[df] = calendars[df][['Week', 'Week Start', 'Week End', 'Week Start Month Number', 'Calendar Week Number', 'Week Sort Number']]
    calendars[df]['Sort ID'] = calendars[df]['Week Sort Number'].astype(str) + '-' + calendars[df]['Calendar Week Number'].astype(str)

In [10]:
master_calendar = pd.DataFrame()

for key, data in calendars.items():
    master_calendar = master_calendar.append(pd.concat([data]))

master_calendar.reset_index(drop=True, inplace=True)
master_calendar

,Week,Week Start,Week End,Week Start Month Number,Calendar Week Number,Week Sort Number,Sort ID
0,2019-08-25/2019-08-31,2019-08-25,2019-08-31,8,35,52,52-35
1,2019-08-18/2019-08-24,2019-08-18,2019-08-24,8,34,51,51-34
2,2019-08-11/2019-08-17,2019-08-11,2019-08-17,8,33,50,50-33
3,2019-08-04/2019-08-10,2019-08-04,2019-08-10,8,32,49,49-32
4,2019-07-28/2019-08-03,2019-07-28,2019-08-03,7,31,48,48-31
...,...,...,...,...,...,...,...
413,2025-10-05/2025-10-11,2025-10-05,2025-10-11,10,41,5,5-41
414,2025-09-28/2025-10-04,2025-09-28,2025-10-04,9,40,4,4-40
415,2025-09-21/2025-09-27,2025-09-21,2025-09-27,9,39,3,3-39
416,2025-09-14/2025-09-20,2025-09-14,2025-09-20,9,38,2,2-38


In [11]:
master_calendar['Sort ID'].duplicated().any()

True

In [12]:
for df in calendars:
    print('Calendar week duplicates:', calendars[df]['Calendar Week Number'].duplicated().any())
    print('Week sort number duplicates:', calendars[df]['Week Sort Number'].duplicated().any())
    print('Sort ID duplicates:', calendars[df]['Sort ID'].duplicated().any())

Calendar week duplicates: False
Week sort number duplicates: False
Sort ID duplicates: False
Calendar week duplicates: True
Week sort number duplicates: False
Sort ID duplicates: False
Calendar week duplicates: False
Week sort number duplicates: False
Sort ID duplicates: False
Calendar week duplicates: False
Week sort number duplicates: False
Sort ID duplicates: False
Calendar week duplicates: False
Week sort number duplicates: False
Sort ID duplicates: False
Calendar week duplicates: False
Week sort number duplicates: False
Sort ID duplicates: False
Calendar week duplicates: True
Week sort number duplicates: False
Sort ID duplicates: False
Calendar week duplicates: False
Week sort number duplicates: False
Sort ID duplicates: False
